Import the Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

Data Preparation

In [ ]:
# load the CSV File
df = pd.read_csv('data/spotify_songs.csv')

# Remove Columns the are Irrelevant
cols_to_drop = ['track_id', 'track_name', 'track_artist', 'track_album_id', 
                'track_album_name', 'track_album_release_date', 'playlist_name', 
                'playlist_id', 'playlist_subgenre']

# Drop the Columns and Remove the Null
new_df = df.drop(columns=cols_to_drop).dropna()

# Label Encoder
le = LabelEncoder()
new_df['playlist_genre'] = le.fit_transform(new_df['playlist_genre'])



       track_popularity  playlist_genre  danceability  energy  key  loudness  \
0                    66               2         0.748   0.916    6    -2.634   
1                    67               2         0.726   0.815   11    -4.969   
2                    70               2         0.675   0.931    1    -3.432   
3                    60               2         0.718   0.930    7    -3.778   
4                    69               2         0.650   0.833    1    -4.672   
...                 ...             ...           ...     ...  ...       ...   
32828                42               0         0.428   0.922    2    -1.814   
32829                20               0         0.522   0.786    0    -4.462   
32830                14               0         0.529   0.821    6    -4.899   
32831                15               0         0.626   0.888    2    -3.361   
32832                27               0         0.603   0.884    5    -4.571   

       mode  speechiness  acousticness 

Scale the Data

In [ ]:
X = new_df.drop('playlist_genre', axis=1)
y = new_df['playlist_genre']

# Split into Training and testing Dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


Train the Model

In [22]:
# Import the Libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Scale the Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10,20, None],
    'min_samples_split' : [2, 5, 20],
    'max_features' : ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=42)

rf_grid_search = GridSearchCV(estimator=rf,
                              param_grid=rf_param_grid,
                              cv=3,
                              scoring='f1_macro',
                              verbose=1,
                              n_jobs=-1)

rf_grid_search.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rf_grid_search.predict(X_test_scaled)

# Print the classification report
print(classification_report(y_test, y_pred, target_names=le.classes_))

Fitting 3 folds for each of 36 candidates, totalling 108 fits
              precision    recall  f1-score   support

         edm       0.65      0.70      0.68      1218
       latin       0.53      0.42      0.47      1033
         pop       0.38      0.33      0.35      1081
         r&b       0.48      0.49      0.49      1031
         rap       0.59      0.65      0.62      1168
        rock       0.68      0.77      0.72      1036

    accuracy                           0.56      6567
   macro avg       0.55      0.56      0.55      6567
weighted avg       0.56      0.56      0.56      6567



Let's try XGBoost


In [19]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimator = 100,
    learning_rate = 0.2,
    max_depth = 6,
    objective='multi:softprob', 
    random_state=42
)

# Train the Model
xgb_model.fit(X_train_scaled, y_train)

# Predict
y_pred2 = xgb_model.predict(X_test_scaled)

# Classification Report
print(classification_report(y_test, y_pred2, target_names=le.classes_))

              precision    recall  f1-score   support

         edm       0.67      0.67      0.67      1218
       latin       0.51      0.41      0.46      1033
         pop       0.39      0.38      0.38      1081
         r&b       0.45      0.50      0.48      1031
         rap       0.56      0.68      0.62      1168
        rock       0.73      0.64      0.68      1036

    accuracy                           0.55      6567
   macro avg       0.55      0.55      0.55      6567
weighted avg       0.55      0.55      0.55      6567



In [ ]:
from sklearn.model_selection import GridSearchCV
# 1. Define the "Grid" of parameters you want to test
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2]
}

# 2. Initialize the model
xgb = XGBClassifier(objective='multi:softprob', random_state=42)

# 3. Setup the Grid Search
# cv=3 means it will test each combination 3 times using different data splits
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, 
                           cv=3, scoring='f1_macro', verbose=1)

# 4. Run the search (this may take a few minutes)
grid_search.fit(X_train_scaled, y_train)

# 5. See the results
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Parameters: {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 100}
Best Score: 0.5557835769236038
